In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gc

# 读取数据

In [ ]:
train = pd.read_csv(r'data/train_20190518.csv',header=None, names = ['label', 'uId', 'adId', 'operTime', 'siteId', 'slotId', 'contentId', 'netType'])
test = pd.read_csv(r'data/test_20190518.csv',header=None, names = ['label', 'uId', 'adId', 'operTime', 'siteId', 'slotId', 'contentId', 'netType'] )
user_info = pd.read_csv(r'data/clean_user_info.csv',header=None, names =['uId', 'age', 'gender', 'city', 'province', 'phoneType', 'carrier'])
ad_info = pd.read_csv(r'data/clean_ad_info.csv',header=None, names =['adId', 'billId', 'primId','creativeType', 'intertype', 'spreadAppId'])
content_info = pd.read_csv(r'data/clean_content_info.csv',header=None, names =['contentId', 'firstClass', 'secondClass'])

In [ ]:
print('train size:',train.shape)
print('test size:',test.shape)
print('user_info size:',user_info.shape)
print('ad_info size:',ad_info.shape)
print('content_info size:',content_info.shape)

# 处理user_info数据

user_info包含很多缺失值，经过观察，只有phoneType这个特征，只有5个缺失值，并且其总共有512个种类，根据常识，使用同种手机的人群特征也是比较类似的，所以根据每一类手机将用户分类，用此类手机中其他特征的众数，来填充此类用户中的缺失值，填充的原则就是取出现次数最多的元素进行填充

如果某一类手机的某个特征中，全部都是缺失值，则使用所有数据中出现次数最多的元素进行填充

In [ ]:
phone_type = user_info['phoneType'].unique().tolist()

# 计算每个特征中，出现次数最多的那个元素
max_total_age = pd.value_counts(user_info['age'])
max_total_age = max_total_age.index[0]

max_total_gender = pd.value_counts(user_info['gender'])
max_total_gender = max_total_gender.index[0]

max_total_city = pd.value_counts(user_info['city'])
max_total_city = max_total_city.index[0]

max_total_province = pd.value_counts(user_info['province'])
max_total_province = max_total_province.index[0]

max_total_phoneType = pd.value_counts(user_info['phoneType'])
max_total_phoneType = max_total_phoneType.index[0]

max_total_carrier = pd.value_counts(user_info['carrier'])
max_total_carrier = max_total_carrier.index[0]

In [ ]:
# 先将phoneType为缺失值的那5个数据用所有数据出现次数最多的元素继续宁填充
tem_int = user_info[user_info['phoneType'].isnull()]['age'].fillna(max_total_age)
user_info.loc[user_info['phoneType'].isnull(),'age'] = tem_int

tem_int = user_info[user_info['phoneType'].isnull()]['gender'].fillna(max_total_gender)
user_info.loc[user_info['phoneType'].isnull(),'gender'] = tem_int

tem_int = user_info[user_info['phoneType'].isnull()]['province'].fillna(max_total_province)
user_info.loc[user_info['phoneType'].isnull(),'province'] = tem_int

tem_int = user_info[user_info['phoneType'].isnull()]['carrier'].fillna(max_total_carrier)
user_info.loc[user_info['phoneType'].isnull(),'carrier'] = tem_int

tem_int = user_info[user_info['phoneType'].isnull()]['phoneType'].fillna(max_total_phoneType)
user_info.loc[user_info['phoneType'].isnull(),'phoneType'] = tem_int

# 用手机类型进行分类，分别处理每个批量的数据
num = 1
for i in phone_type:
    data = user_info[user_info['phoneType'] == i]
    if (data['age'].isnull().sum() == len(data) or 
       data['gender'].isnull().sum() == len(data) or
       data['city'].isnull().sum() == len(data) or
       data['province'].isnull().sum() == len(data) or
       data['carrier'].isnull().sum() == len(data)):
        
        tem_int = user_info[user_info['phoneType'] == i]['age'].fillna(max_total_age)
        user_info.loc[user_info['phoneType'] == i,'age'] = tem_int
        
        tem_int = user_info[user_info['phoneType'] == i]['gender'].fillna(max_total_gender)
        user_info.loc[user_info['phoneType'] == i,'gender'] = tem_int
        
        tem_int = user_info[user_info['phoneType'] == i]['city'].fillna(max_total_city)
        user_info.loc[user_info['phoneType'] == i,'city'] = tem_int
        
        tem_int = user_info[user_info['phoneType'] == i]['province'].fillna(max_total_province)
        user_info.loc[user_info['phoneType'] == i,'province'] = tem_int
        
        tem_int = user_info[user_info['phoneType'] == i]['carrier'].fillna(max_total_carrier)
        user_info.loc[user_info['phoneType'] == i,'carrier'] = tem_int
        
    else:
        max_age = pd.value_counts(data['age'])
        max_age = max_age.index[0]

        max_gender = pd.value_counts(data['gender'])
        max_gender = max_gender.index[0]

        max_city = pd.value_counts(data['city'])
        max_city = max_city.index[0]

        max_province = pd.value_counts(data['province'])
        max_province = max_province.index[0]

        max_carrier = pd.value_counts(data['carrier'])
        max_carrier = max_carrier.index[0]
        
        tem_int = user_info[user_info['phoneType'] == i]['age'].fillna(max_age)
        user_info.loc[user_info['phoneType'] == i,'age'] = tem_int

        tem_int = user_info[user_info['phoneType'] == i]['gender'].fillna(max_gender)
        user_info.loc[user_info['phoneType'] == i,'gender'] = tem_int

        tem_int = user_info[user_info['phoneType'] == i]['city'].fillna(max_city)
        user_info.loc[user_info['phoneType'] == i,'city'] = tem_int

        tem_int = user_info[user_info['phoneType'] == i]['province'].fillna(max_province)
        user_info.loc[user_info['phoneType'] == i,'province'] = tem_int

        tem_int = user_info[user_info['phoneType'] == i]['carrier'].fillna(max_carrier)
        user_info.loc[user_info['phoneType'] == i,'carrier'] = tem_int
    
#     监控处理过程
    print('number of',num)
    num += 1


number of 1
number of 2
number of 3
number of 4
number of 5
number of 6
number of 7
number of 8
number of 9
number of 10
number of 11
number of 12
number of 13
number of 14
number of 15
number of 16
number of 17
number of 18
number of 19
number of 20
number of 21
number of 22
number of 23
number of 24
number of 25
number of 26
number of 27
number of 28
number of 29
number of 30
number of 31
number of 32
number of 33
number of 34
number of 35
number of 36
number of 37
number of 38
number of 39
number of 40
number of 41
number of 42
number of 43
number of 44
number of 45
number of 46
number of 47
number of 48
number of 49
number of 50
number of 51
number of 52
number of 53
number of 54
number of 55
number of 56
number of 57
number of 58
number of 59
number of 60
number of 61
number of 62
number of 63
number of 64
number of 65
number of 66
number of 67
number of 68
number of 69
number of 70
number of 71
number of 72
number of 73
number of 74
number of 75
number of 76
number of 77
number o

In [ ]:
user_info.isnull().any()

In [ ]:
user_info.to_csv('data/clean_user_info.csv',index = False)

# 处理ad_info数据

１．付费方式＂billId＂需要数值化

２．spreadAppId（广告对应的appId）这个特征就有缺失值

解决方案：将spreadAppId作为标签，用前五个特征进行训练，使用KNN算法，预测无标签数据的标签，在验证数据集上，得到了79%的准确率

In [ ]:
# 字符数据离散化 
labels_3 = ad_info['billId'].unique().tolist()
q = 1
for i in labels_3:
    ad_info.loc[ad_info['billId'] ==i,'billId'] = q
    q += 1

In [ ]:
ad_info[ad_info['spreadAppId'].isnull()]['primId'].unique()

In [ ]:
X_data = ad_info[ad_info['spreadAppId'].notnull()]
y = X_data['spreadAppId']
X =  X_data.drop(['spreadAppId'],axis = 1)

X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2)
knn=KNeighborsClassifier(n_neighbors=2,weights='distance')
knn.fit(X_train,Y_train)
y_pred = knn.predict(X_test)
print('accuracy of KNN',accuracy_score(Y_test, y_pred))

null_data = ad_info[ad_info['spreadAppId'].isnull()]
null_data = null_data.drop(['spreadAppId'],axis = 1)
y_pred_null = knn.predict(null_data)
null_data['spreadAppId'] = y_pred_null

In [ ]:
ad_info = pd.concat([X_data,null_data])
cad_info = clean_ad_info.sort_index()
ad_info.to_csv('data/clean_ad_info.csv',index = False)

# 处理content_info数据

对于content info文件，特征secondClass是对firstClass的一个更详细的分类。对于每一个firstClass，同一个firstClass下对应的secondClass是固定且唯一的。并且文件中只有secondClass存在缺失值，因此先将数据根据firstClass进行分类，再在每一个小类中，用seccondClass出现次数最多的元素填充缺失值

再将字符串特征转换成数值特征（从１开始）

In [ ]:
first_class = content_info['firstClass'].unique().tolist()

for i in first_class:
    data = content_info[content_info['firstClass'] == i]   
    
    if (data['secondClass'].isnull().sum() == len(data)):   
        content_info.loc[content_info['firstClass'] == i,'secondClass'] = i
        
    else:
        max_secondclass = pd.value_counts(data['secondClass'])
        max_secondclass = max_secondclass.index[0]

        tem_int = content_info[content_info['firstClass'] == i]['secondClass'].fillna(max_secondclass)
        content_info.loc[content_info['firstClass'] == i,'secondClass'] = tem_int
        
        
content_info["firstClass"] = pd.factorize(content_info["firstClass"])[0].astype(np.uint64)
content_info["secondClass"] = pd.factorize(content_info["secondClass"])[0].astype(np.uint64)

content_info.loc[content_info['firstClass'] ==0,'firstClass'] = 23
content_info.loc[content_info['secondClass'] ==0,'secondClass'] = 89

In [ ]:
content_info.to_csv('clean_contentId_info.csv',index = False)

# 查看训练数据与测试数据

１．训练数据中无缺失值，无需处理

２．测试数据中，＂contentId＂存在缺失值，数目并不多，采用众数填充

In [ ]:
train.isnull().any()

In [ ]:
test.isnull().any()

In [92]:
max_contendId = pd.value_counts(test['contentId'])
max_contendId = max_contendId.index[0]

tem_int = test[test['contentId'].isnull()]['contentId'].fillna(max_contendId)
test.loc[test['contentId'].isnull(),'contentId'] = tem_int

In [93]:
test.isnull().any()

label        False
uId          False
adId         False
operTime     False
siteId       False
slotId       False
contentId    False
netType      False
dtype: bool

In [94]:
test.to_csv('data/clean_test.csv',index = False)